# Laboratorio de regresión - 5

|                |   |
:----------------|---|
| **Nombre**     | Eugenio Hernández Aranda  |
| **Fecha**      | 16/02/2026  |
| **Expediente** | 756225  |

## Validación

Hemos estado usando `train_test_split` en nuestros modelos anteriores.

¿Por qué?

Para poder entrenar nuestros modelos con una parte de los datos mientras que podemos hacer el test para ver si funciona con lo que resta de los datos.

Si la muestra es un subset de la población y queremos generalizar sobre la población, ¿no sería mejor utilizar todos los datos al entrenar un modelo?

No, porque al usar todos los datos nuestro modelo estaríamos sesgando los resultados, esto debido a que no puedes saber si un modelo funciona probando los mismos datos con los que fue entrenado.

El propósito de volver a muestrear dentro de nuestro dataset es tener una idea de qué tan buena podría ser la generalización de nuestro modelo. Imagina un dataset ya separado en dos mitades. Utilizas la primera mitad para entrenar el modelo y pruebas en la segunda mitad; la segunda mitad eran datos invisibles para el modelo al momento de entrenar. Esto nos lleva a tres escenario típicos:

1. Si el modelo hace buenas predicciones en la segunda mitad, significa que la primera mitad era "suficiente" para generalizar.
2. Si el modelo no hace buenas predicciones en la segunda mitad, pero sí en la primera mitad, podría ser que había información importante en la segunda mitad que debió haber sido tomada en cuenta al entrenar, o un problema de overfitting.
3. Si el modelo no hace buenas predicciones en la segunda mitad, y tampoco en la primera mitad, se tendrían que revisar los factores y/o el modelo seleccionado.

El caso ideal sería el 1, pero por estadística los errores y varianzas tienen como entrada el número de muestas, por lo que tenemos menos seguridad de nuestros resutados al usar menos muestras. Si vemos que el modelo generaliza bien podemos unir de nuevo el dataset y entrenar sobre el dataset completo.

En el caso 2 está el problema de que no podemos saber qué información es necesaria para el entrenamiento apropiado del modelo; esto nos lleva a pensar que debemos usar el dataset completo para entrenar, pero esto nos lleva al mismo problema de no saber si el modelo puede generalizar.

El problema sólo incrementa si se tienen hiperparámetros en el modelo (e.g. $\lambda$ en regularización).

## Leave-One-Out Cross Validation

Este método de validación es una colección de $n$ `train-test-split`. Teniendo un dataset de $n$ muestras, la lógica es:
1. Saca una muestra del dataset.
2. Entrena tu modelo con las $n-1$ muestras.
3. Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación.
4. Regresa la muestra al dataset.
5. Repite 1-4 con muestras diferentes hasta haber hecho el procedimiento $n$ veces para $n$ muestras.
6. Calcula la media y desviación estándar de los métricos guardados.

Con los resultados del proceso de validación podemos saber qué tan bueno podría ser el modelo seleccionado con los datos (con/sin transformaciones).

### Ejercicio 1

Utiliza el dataset `Motor Trend Car Road Tests`. Elimina la columna `model` y entrena 32 modelos diferentes utilizando Leave-One-Out Cross Validation con target `mpg`. Utiliza MSE como métrico.

In [1]:
from google.colab import files
uploaded = files.upload()

Saving Motor Trend Car Road Tests.xlsx to Motor Trend Car Road Tests.xlsx


In [19]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

In [14]:
df = pd.read_excel('Motor Trend Car Road Tests.xlsx', )
df

,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2
5,Valiant,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1
6,Duster 360,14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4
7,Merc 240D,24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2
8,Merc 230,22.8,4,140.8,95,3.92,3.150,22.90,1,0,4,2
9,Merc 280,19.2,6,167.6,123,3.92,3.440,18.30,1,0,4,4


In [15]:
#1. Saca una muestra del dataset.
#2. Entrena tu modelo con las $n-1$ muestras.
#3. Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación.
#4. Regresa la muestra al dataset.
#5. Repite 1-4 con muestras diferentes hasta haber hecho el procedimiento $n$ veces para $n$ muestras.
#6. Calcula la media y desviación estándar de los métricos guardados.
indices = np.arange(0,32)
indices
df_sin_model = df.drop(index=[0])
df_sin_model

,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2
5,Valiant,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1
6,Duster 360,14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4
7,Merc 240D,24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2
8,Merc 230,22.8,4,140.8,95,3.92,3.150,22.90,1,0,4,2
9,Merc 280,19.2,6,167.6,123,3.92,3.440,18.30,1,0,4,4
10,Merc 280C,17.8,6,167.6,123,3.92,3.440,18.90,1,0,4,4


In [20]:
y_train = df['qsec'].to_numpy().reshape(-1,1)
am = df['am'].to_numpy().reshape(-1,1)
carb = df['carb'].to_numpy().reshape(-1,1)
gear= df['gear'].to_numpy().reshape(-1,1)
disp= df['disp'].to_numpy().reshape(-1,1)
hp = df['hp'].to_numpy().reshape(-1,1)
wt = df['wt'].to_numpy().reshape(-1,1)
mpg = df['mpg'].to_numpy().reshape(-1,1)
cyl = df['cyl'].to_numpy().reshape(-1,1)
drat = df['drat'].to_numpy().reshape(-1,1)
vs = df['vs'].to_numpy().reshape(-1,1)
X_train_to_scale = np.hstack((mpg, cyl, disp, hp, drat, am, gear, carb))

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_to_scale)

X_train = np.hstack((X_train_scaled, wt, vs))
X_train

array([[ 0.15329914, -0.10666772, -0.57975032, -0.54365487,  0.57659448,
         1.20894105,  0.43033148,  0.74696708,  2.62      ,  0.        ],
       [ 0.15329914, -0.10666772, -0.57975032, -0.54365487,  0.57659448,
         1.20894105,  0.43033148,  0.74696708,  2.875     ,  0.        ],
       [ 0.4567366 , -1.24445674, -1.00602601, -0.7955699 ,  0.48158406,
         1.20894105,  0.43033148, -1.14010764,  2.32      ,  1.        ],
       [ 0.22072968, -0.10666772,  0.22361542, -0.54365487, -0.98157639,
        -0.82717019, -0.94672926, -1.14010764,  3.215     ,  1.        ],
       [-0.23442651,  1.0311213 ,  1.05977159,  0.41954967, -0.84856181,
        -0.82717019, -0.94672926, -0.51108274,  3.44      ,  0.        ],
       [-0.33557233, -0.10666772, -0.0469057 , -0.61774753, -1.58964307,
        -0.82717019, -0.94672926, -1.14010764,  3.46      ,  1.        ],
       [-0.97616253,  1.0311213 ,  1.05977159,  1.45684686, -0.7345493 ,
        -0.82717019, -0.94672926,  0.74696708

In [24]:
X_train = sm.add_constant(X_train)
model1=sm.OLS(y_train, X_train) # -> Modela minimos cuadrados ordinarios con todas las variables
results=model1.fit() #-> Optimiza
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.875
Model:                            OLS   Adj. R-squared:                  0.815
Method:                 Least Squares   F-statistic:                     14.66
Date:                Mon, 16 Feb 2026   Prob (F-statistic):           2.44e-07
Time:                        23:39:50   Log-Likelihood:                -30.242
No. Observations:                  32   AIC:                             82.48
Df Residuals:                      21   BIC:                             98.61
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         12.6103      1.596      7.903      0.0

In [28]:
# Evaluamos en la fila 0
df_eval = df.iloc[0]
model1.predict()

ValueError: shapes (32,11) and (12,) not aligned: 11 (dim 1) != 12 (dim 0)

Interpreta.

## K-Folds Cross-Validation

El dataset `Motor Trend Car Road Tests` sólo tiene 32 muestras, y utilizar un modelo sencillo de regresión múltiple hace que usar LOOCV sea muy rápido. El dataset `California Housing` tiene $20640$ muestras para $9$ columnas, entonces realizar un ajuste sobre una transformación o sobre el modelo y luego calcular el impacto esperado podría tomar más tiempo.

La solución propuesta es dividir el dataset en *k* folds (partes iguales), ajustar en *k-1* folds y probar en el restante.

### Ejercicio 2
Utiliza el dataset `California Housing` y haz K-folds Cross Validation con 10 folds. Utiliza el MSE como métrico.

In [ ]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print("Dataset Shape:", housing.data.shape, housing.target.shape)
print("Dataset Features:", housing.feature_names)
print("Dataset Target:", housing.target_names)
X = housing.data
y = housing.target

Interpreta.

## Referencia

James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3